**Upgrade SDK**

In [0]:
%pip install --upgrade databricks-sdk
%restart_python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.8/749.8 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: databricks-sdk
    Found existing installation: databricks-sdk 0.49.0
    Not uninstalling databricks-sdk at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-08d2f129-1362-4611-a80e-e50d7fd03a94
    Can't uninstall 'databricks-sdk'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip show databricks-sdk | grep -oP '(?<=Version: )\S+'

0.69.0


##### Alerting Demo
###### This notebook is a demo that allows you to:
###### - Create new queries and alerts
###### - Retrieve SQL queries and alerts
###### - Work via **Databricks SDK** 
###### - Print summaries for queries and alerts

**No need to set Databricks host and access token if you are working from within Databricks**

In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

**Set up some folks for notifying, rest of constants will come naturally later**

In [0]:
MY_EMAIL='vanagnostopoulos@aegeanair.com'
DESTINATION_EMAILS = ['konstantinos.psaroulakis@agileactors.com', "vasilis.anagnostopoulos@agileactors.com"]

**Cleanup test artefacts from previous runs**

**Cleanup test queries**

In [0]:
test_my_queries = [
    some_query for some_query in w.queries.list() if some_query.display_name.startswith('mysecond_tutorial_query')
]

print(list(test_my_queries))

for query in test_my_queries:
    w.queries.delete(id=query.id)

test_my_queries = [
    some_query for some_query in w.queries.list() if some_query.display_name.startswith('mysecond_tutorial_query')
]

assert test_my_queries == []

[]


**Cleanup test alerts**

In [0]:
some_alerts = w.alerts.list()

test_my_alerts = [
    some_alert for some_alert in some_alerts if some_alert.display_name.startswith('mysecond_tutorial_alert')
]

print(list(test_my_alerts))

for some_alert in test_my_alerts:
    w.alerts.delete(id=some_alert.id)

assert [
    some_alert for some_alert in w.alerts.list() if some_alert.display_name.startswith('mysecond_tutorial_alert')
] == []

[]


**Cleanup test schedules**

In [0]:
some_jobs = w.jobs.list()
test_my_schedules = [
    some_job for some_job in some_jobs if some_job.settings.name.startswith('mysecond_tutorial_schedule')
]

for some_job in test_my_schedules:
    print(some_job)
    w.jobs.delete(some_job.job_id)


assert [
    some_job for some_job in w.jobs.list() if some_job.settings.name.startswith('mysecond_tutorial_schedule')
] == []



**Cleanup test destinations**

In [0]:
some_notification_destinations = list(w.notification_destinations.list())
print(some_notification_destinations)

for some_notification_destination in some_notification_destinations:
    print(some_notification_destination)
    if some_notification_destination.display_name.startswith('mysecond_tutorial_destirnation'):
        w.notification_destinations.delete(id=some_notification_destination.id)

some_notification_destinations = list(w.notification_destinations.list())
print(some_notification_destinations)

[ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='Mavroeidi Konstantina', id='02bcac4b-eaa3-4ae8-8fc6-5ee83e536c6a'), ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='Evangelatos_Pavlos', id='037c3170-a93d-42e0-a229-65e73cce72e7'), ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='Kapodistria_Angeliki', id='068452e9-4a7f-4edf-9b6c-6f029e8b2088'), ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='DevOps_Email_Kapodistria_Angeliki', id='0841c18a-bc23-47fb-8703-1cec7276d39e'), ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='Christos Grigoras', id='090dd30e-f716-48ac-9fe2-f3d482e578c6'), ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='Unlogged_Hours_Margarita Vitoropoulou', id='094c44cc-aa

#### Retrieve sources for query creation, it should be a warehouse for executing the query

In [0]:

srcs = w.data_sources.list()
print(srcs)
simple_warehouse_source = [
    some_warehouse for some_warehouse in srcs if some_warehouse.name == 'Simple SQL Warehouse'
][0]

simple_warehouse_source

[DataSource(id='1d54d818-2770-4209-9373-f53976e078bc', name='Dashboard SQL Warehouse', pause_reason=None, paused=0, supports_auto_limit=True, syntax='sql', type='databricks_internal', view_only=False, warehouse_id='fe0b9d9b3fc4b4e6'), DataSource(id='06632428-fd44-4262-8e6c-9481b011d881', name='DSL SQL Warehouse', pause_reason=None, paused=0, supports_auto_limit=True, syntax='sql', type='databricks_internal', view_only=False, warehouse_id='18a70bc6aafdd1e8'), DataSource(id='f602f2b0-8979-474d-9160-137a2441afbc', name='DSL_AI', pause_reason=None, paused=0, supports_auto_limit=True, syntax='sql', type='databricks_internal', view_only=False, warehouse_id='df19317276ad901d'), DataSource(id='1356dd4c-38c4-4dba-9608-4e674ca12497', name='iTeam SQL Warehouse', pause_reason=None, paused=0, supports_auto_limit=True, syntax='sql', type='databricks_internal', view_only=False, warehouse_id='c60c29392dd5e65d'), DataSource(id='61f00cd2-8b26-4aae-88fb-69925f54d54e', name='Medium SQL Warehouse', pause_r

DataSource(id='971b53be-96ca-49f9-b4a7-690338c88422', name='Simple SQL Warehouse', pause_reason=None, paused=0, supports_auto_limit=True, syntax='sql', type='databricks_internal', view_only=False, warehouse_id='de26e359b8a57e8a')

**Create test query**

_See here https://databricks-sdk-py.readthedocs.io/en/latest/workspace/sql/queries.html_


In [0]:
from databricks.sdk.service import sql

srcs = w.data_sources.list()

test_sql_query = """
with tutorial_data as (
SELECT explode(sequence(1, 100)) AS sequence_number, random() as some_weight
) select count(*) as count_random_data2 from tutorial_data where some_weight > 0.5;
"""

test_my_query = w.queries.create(
    query=sql.CreateQueryRequestQuery(
        display_name="mysecond_tutorial_query",
        warehouse_id=simple_warehouse_source.warehouse_id,
        description="test query provided for me",
        query_text=test_sql_query,
    )
)

# cleanup
# w.queries.delete(id=test_my_query.id)

**Summary of my queries**

In [0]:
[
    some_query.display_name for some_query in w.queries.list() if some_query.owner_user_name==MY_EMAIL
]

['atuturoail_querya', 'mysecond_tutorial_query']

**Updates on test query**

In [0]:
updated_test_sql_query = """
with tutorial_data as (
SELECT explode(sequence(1, 200)) AS sequence_number, random() as some_weight
) select count(*) as count_random_data2 from tutorial_data where some_weight > 0.5;
"""

updated_test_my_query = w.queries.update(
    id=test_my_query.id,
    query=sql.UpdateQueryRequestQuery(
        display_name=f"mysecond_tutorial_query2",
        description="updated test query provided for me",
        query_text=updated_test_sql_query,
    ),
    update_mask="display_name,description,query_text",
)

# cleanup
# w.queries.delete(id=updated_test_my_query.id)

**Assert the update**

In [0]:
assert test_my_query.id == updated_test_my_query.id

updated_test_my_query

Query(apply_auto_limit=None, catalog=None, create_time='2025-10-21T14:25:43Z', description='updated test query provided for me', display_name='mysecond_tutorial_query2', id='de8b0c1e-7343-4723-ae03-15d386238db5', last_modifier_user_name='vanagnostopoulos@aegeanair.com', lifecycle_state=<LifecycleState.ACTIVE: 'ACTIVE'>, owner_user_name='vanagnostopoulos@aegeanair.com', parameters=[], parent_path='/Workspace/Users/vanagnostopoulos@aegeanair.com', query_text='\nwith tutorial_data as (\nSELECT explode(sequence(1, 200)) AS sequence_number, random() as some_weight\n) select count(*) as count_random_data2 from tutorial_data where some_weight > 0.5;\n', run_as_mode=<RunAsMode.OWNER: 'OWNER'>, schema=None, tags=None, update_time='2025-10-21T14:26:24Z', warehouse_id='de26e359b8a57e8a')

**Create test alert**

viewable at 
https://adb-4600033890908005.5.azuredatabricks.net/sql/alerts?o=4600033890908005&order=-created_at&order_by=created_at+desc&page=1&page_size=20


In [0]:
test_my_alert = w.alerts.create(
    alert=sql.CreateAlertRequestAlert(
        condition=sql.AlertCondition(
            operand=sql.AlertConditionOperand(column=sql.AlertOperandColumn(name="count_random_data2")),
            op=sql.AlertOperator.GREATER_THAN_OR_EQUAL,
            threshold=sql.AlertConditionThreshold(value=sql.AlertOperandValue(double_value=100)),
        ),
        display_name="mysecond_tutorial_alert",
        query_id=updated_test_my_query.id,
        seconds_to_retrigger=3600
    )
)

# cleanup
# w.alerts.delete(id=test_my_alert.id)

**Summary of my alerts**

In [0]:
[
    some_alert for some_alert in w.alerts.list() if some_alert.owner_user_name==MY_EMAIL
]

[ListAlertsResponseAlert(condition=AlertCondition(empty_result_state=<AlertState.UNKNOWN: 'UNKNOWN'>, op=<AlertOperator.GREATER_THAN: 'GREATER_THAN'>, operand=AlertConditionOperand(column=AlertOperandColumn(name='count_random_data')), threshold=AlertConditionThreshold(value=AlertOperandValue(bool_value=None, double_value=None, string_value='50'))), create_time='2025-10-21T14:16:33Z', custom_body=None, custom_subject=None, display_name='vasilis_alert', id='c81a9c46-91a6-46b7-b734-d6e1b73e44ff', lifecycle_state=<LifecycleState.ACTIVE: 'ACTIVE'>, notify_on_ok=False, owner_user_name='vanagnostopoulos@aegeanair.com', query_id='81b9ec82-674f-4d46-ad05-78cc5dd800b7', seconds_to_retrigger=0, state=<AlertState.TRIGGERED: 'TRIGGERED'>, trigger_time='2025-10-21T14:28:23Z', update_time='2025-10-21T14:28:23Z'),
 ListAlertsResponseAlert(condition=AlertCondition(empty_result_state=None, op=<AlertOperator.GREATER_THAN_OR_EQUAL: 'GREATER_THAN_OR_EQUAL'>, operand=AlertConditionOperand(column=AlertOperan

**Create some test destinations**

In [0]:
from databricks.sdk.service import settings

notification_destination_no0 = w.notification_destinations.create(
    config = settings.Config(
        email=settings.EmailConfig(addresses=[DESTINATION_EMAILS[0]])
    ),
    display_name="mysecond_tutorial_destination_no0",
)

notification_destination_no1 = w.notification_destinations.create(
    config = settings.Config(
        email=settings.EmailConfig(
            addresses=[DESTINATION_EMAILS[1]]
        )
    ),
    display_name="mysecond_tutorial_destination_no1",
)

# cleanup
# w.notification_destinations.delete(id=notification_destination_no0.id)
# w.notification_destinations.delete(id=notification_destination_no1.id)

**Let's see the notification destinations**

In [0]:
[
    some_notification_destination 
    for some_notification_destination in w.notification_destinations.list() 
    if some_notification_destination.display_name.startswith('mysecond_tutorial_destination')
]

[ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='mysecond_tutorial_destination_no1', id='9c9bd3b1-f021-42cb-94e7-d9183151dec5'),
 ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='mysecond_tutorial_destination_no0', id='e7804fe8-e13b-4ad1-b48f-33bdacc44a9d')]

**Update the notification destinations**

In [0]:
notification_destination_no0_updated = w.notification_destinations.update(id=notification_destination_no0.id, display_name="mysecond_tutorial")

[
    some_notification_destination 
    for some_notification_destination in w.notification_destinations.list() 
    if some_notification_destination.display_name.startswith('mysecond_tutorial')
]

[ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='mysecond_tutorial_destination_no1', id='9c9bd3b1-f021-42cb-94e7-d9183151dec5'),
 ListNotificationDestinationsResult(destination_type=<DestinationType.EMAIL: 'EMAIL'>, display_name='mysecond_tutorial', id='e7804fe8-e13b-4ad1-b48f-33bdacc44a9d')]

##  Death by accounting
**We need an sql warehouse to run the alert computation**

In [0]:
some_warehouses = w.warehouses.list()
simple_warehouse = [
    some_warehouse for some_warehouse in some_warehouses if some_warehouse.name == 'Simple SQL Warehouse'
][0]

simple_warehouse

EndpointInfo(auto_stop_mins=30, channel=Channel(dbsql_version=None, name=<ChannelName.CHANNEL_NAME_CURRENT: 'CHANNEL_NAME_CURRENT'>), cluster_size='2X-Small', creator_name='ioannis.tselepis@aegeanair.com', enable_photon=True, enable_serverless_compute=False, health=EndpointHealth(details=None, failure_reason=None, message=None, status=<Status.HEALTHY: 'HEALTHY'>, summary=None), id='de26e359b8a57e8a', instance_profile_arn=None, jdbc_url='jdbc:spark://adb-4600033890908005.5.azuredatabricks.net:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/de26e359b8a57e8a;', max_num_clusters=1, min_num_clusters=1, name='Simple SQL Warehouse', num_active_sessions=None, num_clusters=1, odbc_params=OdbcParams(hostname='adb-4600033890908005.5.azuredatabricks.net', path='/sql/1.0/warehouses/de26e359b8a57e8a', port=443, protocol='https'), spot_instance_policy=<SpotInstancePolicy.COST_OPTIMIZED: 'COST_OPTIMIZED'>, state=<State.RUNNING: 'RUNNING'>, tags=EndpointTags(custom_tags=[En

**Let's create a task to host the alert computation**

In [0]:
from databricks.sdk.service import jobs

sql_alert_task = jobs.SqlTask(
    warehouse_id = simple_warehouse.id,
    alert=jobs.SqlTaskAlert(
        alert_id=test_my_alert.id,
        subscriptions=[
            jobs.SqlTaskSubscription(
                destination_id=notification_destination_no1.id
            )
        ],
    )
)

**Setup a job to run the alert computational task**

In [0]:
from databricks.sdk import WorkspaceClient

cron_schedule="0 45 15 * * ?"

test_my_schedule = w.jobs.create(
    name="mysecond_tutorial_schedule",
    schedule=jobs.CronSchedule(quartz_cron_expression=cron_schedule, timezone_id='Europe/Athens',pause_status=jobs.PauseStatus.PAUSED), #should be UNPAUSED to start
    tasks=[
        jobs.Task(
            sql_task=sql_alert_task,
            task_key="mysecond_tutorial_alert_query_task",
            timeout_seconds=0,
        )
    ],
)

# cleanup
# w.jobs.delete(job_id=test_my_schedule.job_id)

**Show my jobs**

In [0]:
[
    some_job for some_job in w.jobs.list() if some_job.creator_user_name==MY_EMAIL
]

[BaseJob(created_time=1761057164770, creator_user_name='vanagnostopoulos@aegeanair.com', effective_budget_policy_id=None, effective_usage_policy_id=None, has_more=None, job_id=602077498374685, settings=JobSettings(budget_policy_id=None, continuous=None, deployment=None, description=None, edit_mode=None, email_notifications=JobEmailNotifications(no_alert_for_skipped_runs=None, on_duration_warning_threshold_exceeded=None, on_failure=None, on_start=None, on_streaming_backlog_exceeded=None, on_success=None), environments=[], format=<Format.MULTI_TASK: 'MULTI_TASK'>, git_source=None, health=None, job_clusters=[], max_concurrent_runs=1, name='mysecond_tutorial_schedule', notification_settings=None, parameters=[], performance_target=None, queue=QueueSettings(enabled=True), run_as=None, schedule=CronSchedule(quartz_cron_expression='0 45 15 * * ?', timezone_id='Europe/Athens', pause_status=<PauseStatus.PAUSED: 'PAUSED'>), tags=None, tasks=[], timeout_seconds=0, trigger=None, usage_policy_id=Non

**Update job for alerting**

In [0]:
w.jobs.delete(job_id=test_my_schedule.job_id)
test_my_schedule = w.jobs.create(
    name="mysecond_tutorial_schedule_updated",
    schedule=jobs.CronSchedule(quartz_cron_expression=cron_schedule, timezone_id='Europe/Athens',pause_status=jobs.PauseStatus.PAUSED), #should be UNPAUSED to start
    tasks=[
        jobs.Task(
            sql_task=sql_alert_task,
            task_key="mysecond_tutorial_alert_query_task",
            timeout_seconds=0,
        )
    ],
)

[
    some_job for some_job in w.jobs.list() if some_job.creator_user_name==MY_EMAIL
]

[BaseJob(created_time=1761057234004, creator_user_name='vanagnostopoulos@aegeanair.com', effective_budget_policy_id=None, effective_usage_policy_id=None, has_more=None, job_id=567927261484173, settings=JobSettings(budget_policy_id=None, continuous=None, deployment=None, description=None, edit_mode=None, email_notifications=JobEmailNotifications(no_alert_for_skipped_runs=None, on_duration_warning_threshold_exceeded=None, on_failure=None, on_start=None, on_streaming_backlog_exceeded=None, on_success=None), environments=[], format=<Format.MULTI_TASK: 'MULTI_TASK'>, git_source=None, health=None, job_clusters=[], max_concurrent_runs=1, name='mysecond_tutorial_schedule_updated', notification_settings=None, parameters=[], performance_target=None, queue=QueueSettings(enabled=True), run_as=None, schedule=CronSchedule(quartz_cron_expression='0 45 15 * * ?', timezone_id='Europe/Athens', pause_status=<PauseStatus.PAUSED: 'PAUSED'>), tags=None, tasks=[], timeout_seconds=0, trigger=None, usage_polic